In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

import torch

from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=4


In [2]:
quantized_model = AutoModelForCausalLM.from_pretrained("BlackSamorez/Llama-2-7b-AQLM-6288ppl-hf", trust_remote_code=True, low_cpu_mem_usage=True, torch_dtype="auto").cuda()
fp16_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", torch_dtype="auto").cuda()
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

configuration_llama_aqlm.py:   0%|          | 0.00/421 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/BlackSamorez/Llama-2-7b-AQLM-6288ppl-hf:
- configuration_llama_aqlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_llama_aqlm.py:   0%|          | 0.00/65.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/BlackSamorez/Llama-2-7b-AQLM-6288ppl-hf:
- modeling_llama_aqlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/home/blacksamorez/quip-sharp/.conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
output = quantized_model.generate(tokenizer("Hi", return_tensors="pt")["input_ids"].cuda(), max_new_tokens=100)
print(tokenizer.decode(output[0]))
output = fp16_model.generate(tokenizer("Hi", return_tensors="pt")["input_ids"].cuda(), max_new_tokens=100)
print(tokenizer.decode(output[0]))

<s> Hi, I'm a newbie here. I'm looking for a good place to learn about the basics of programming.
I'm looking for a good place to learn about the basics of programming.
I'm looking for a good place to learn about the basics of programming. I'm looking for a good place to learn about the basics of programming. I'm looking for a good place to learn about the basics of programming. I'm looking
<s> Hi, I’m Jesse. I’m a 24 year old writer and artist from Canada. I have a degree in English Literature from the University of Toronto. I’m a member of the LGBTQ+ community and a huge supporter of it. I’m a feminist and an intersectional feminist. I’m an atheist. I’m a panromantic asexual. I’m asexual, but I’m


In [4]:
%%time
output = quantized_model.generate(tokenizer("Hi", return_tensors="pt")["input_ids"].cuda(), max_new_tokens=100)

CPU times: user 4.2 s, sys: 3.48 ms, total: 4.2 s
Wall time: 4.2 s


In [5]:
%%time
output = fp16_model.generate(tokenizer("Hi", return_tensors="pt")["input_ids"].cuda(), max_new_tokens=100)

CPU times: user 2.91 s, sys: 12.1 ms, total: 2.93 s
Wall time: 2.92 s
